# __Assisted Practice: Training Deep Neural Networks on TensorFlow__
Building Deep Neural Networks on TensorFlow refers to the process of designing and constructing neural network models using the TensorFlow framework. This involves defining the architecture of the neural network, selecting appropriate layers and activation functions, specifying the optimization algorithm, and training the model using data.

Let's understand how to build and train a neural network using TensorFlow.



## Steps to be followed:
1. Import the required libraries
2. Load and inspect the data
3. Build the model
4. Train the model

### Step 1: Import the required libraries

- Import Pandas and NumPy packages.
- Import the TensorFlow package, which is used for text-based applications, image recognition, voice search, and many more.
- Import the Python package cv2, which is used for computer vision and image processing.
- Import the Python package matplotlib, which sets the padding between and around the subplots as well as the figure size.
- Import necessary libraries and modules for building a deep learning model using TensorFlow. It includes modules for convolutional and pooling layers, dropout, flattening, and dense layers.
- Import other libraries for data manipulation, visualization, and image processing.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense


import numpy as np
import os
import sys


import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib

from dnn_helpers import get_all_callbacks

### Step 2: Load and inspect the data


- Load the Boston Housing dataset using the **keras.datasets.boston_housing.load_data()** function.
- Split the dataset into two sets: the training set **train_features** and **train_labels** and the testing set **test_features** and **test_labels**.
- The training set contains input features (for example, crime rate and number of rooms) and corresponding target labels (for example, the median value of owner-occupied homes).
- The testing set is used to evaluate the trained model's performance.


In [2]:

(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

- The mean **train_mean** and standard deviation **train_std** are calculated along the columns **axis=0** of the **train_features** array.
- Then, the **train_features** array is standardized by subtracting the mean and dividing the resultant by the standard deviation.
- This standardization process ensures that the features have a zero mean and unit variance, which can help improve the training performance and convergence of the model.

In [3]:
train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)
train_features = (train_features - train_mean) / train_std

In [4]:
train_features[0], train_features.shape

(array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
        -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
         1.14850044,  0.44807713,  0.8252202 ]),
 (404, 13))

 __Observation:__


- Here, we can see a few Boston housing datasets.
- The given array represents a multi-dimensional array containing numerical values.
- Each row in the array corresponds to a set of features or data points, while each column represents a specific feature or variable.

### Step 3: Build the Model
Building the neural network requires:
- Configuring the layers of the model and compiling the model.
- Stacking a few layers together using **keras.Sequential**.
- Configuring the loss function, optimizer, and metrics to monitor.
These are added during the model's compile step.



Terminologies:
- The **Loss** function measures how accurate the model is during training; we want to minimize this with the optimizer.
- One must **Optimize** how the model is updated based on the data it sees and its loss function.
- **Metrics** are used to monitor the training and testing steps.

In [5]:

def build_model():
    model = keras.Sequential([
        keras.layers.Input(shape=[len(train_features[0])]),
        Dense(20, activation=tf.nn.relu),
        Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.Adam(),
                  loss='mae',
                  metrics=['mean_absolute_error'])
    return model


### Step 4: Train the model
Training the neural network model requires the following steps:


- Define a custom callback class **PrintDot**, which prints a dot for every epoch during training.

- Create an instance of the model using the **build_model** function.

- Create an instance of EarlyStopping callback, which monitors the validation loss and stops training if it doesn't improve after a certain number of epochs (specified by patience).

- Train the model using the training features and labels. It runs for 200 epochs, with a validation split of 0.1 (10% of the training data used for validation). The callbacks parameter includes **early_stop** and **PrintDot** callbacks.

- Create a Pandas **DataFrame hist** from the history object returned by the model.fit method. It contains the recorded training and validation metrics.

- Extract the last value of the validation mean absolute error (MAE) from the hist DataFrame and assign it to the variable mae_final.

- Print the final MAE on the validation set, rounded to three decimal places.

In [6]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):

        if epoch % 200 == 0:
            print('\nEpoch=', epoch, 'Train loss=', logs['loss'])
        else:
            print(".", end="")

model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_features, train_labels, epochs=5_000, verbose=0, validation_split = 0.2,
                    callbacks=[early_stop, PrintDot()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

mae_final = float(hist['val_mean_absolute_error'].tail(1).iloc[0])

print()
print('Final  Mean absolute  Error on validation set: {}'.format(round(mae_final, 3)))



Epoch= 0 Train loss= 21.985206604003906
.......................................................................................................................................................................................................
Epoch= 200 Train loss= 2.4427618980407715
.......................................................................................................................................................................................................
Epoch= 400 Train loss= 1.9485777616500854
.......................................................................................................................................................................................................
Epoch= 600 Train loss= 1.8125717639923096
......................
Final  Mean absolute  Error on validation set: 2.57


**Observation:**

As shown, the final mean absolute error on the validation set is 2.596.

- Normalize the test features based on the mean and standard deviation of the training set.
- Evaluate the model's performance on the normalized test features and prints the mean absolute error (MAE) on the test set.

In [7]:
test_features_norm = (test_features - train_mean) / train_std
mae,  _ = model.evaluate(test_features_norm, test_labels)
#rmae = np.sqrt(mae)
print(' Mean absolute  Error on test set: {}'.format(round(mae, 3)))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.3342 - mean_absolute_error: 2.3342
 Mean absolute  Error on test set: 2.594


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 20)             │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 905 (3.54 KB)

 Trainable params: 301 (1.18 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 604 (2.36 KB)

**Observation:**

The output indicates the following:

- The evaluation was performed on 4 data points.
- The loss value (mean squared error) on the test set is 3.1237.
- The mean absolute error on the test set is also 3.1237.
- The mean absolute error, when rounded, is 3.124.

In summary, the model achieved a loss value of 3.1237 and a mean absolute error of 3.1237, which translates to a mean absolute error of approximately 3.124.